In [ ]:
import json
import pandas as pd
import nltk
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [ ]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rohit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Loading the dataset

In [ ]:
data = json.load(open('E:/Course work/NLP/contract-nli/contract-nli/train.json'))
df = pd.DataFrame(data["documents"])

In [ ]:
df.head()

,id,file_name,text,spans,annotation_sets,document_type,url
0,34,Annex E_Non-Disclosure and Confidentiality Agr...,NON-DISCLOSURE AND CONFIDENTIALITY AGREEMENT\n...,"[[0, 44], [45, 132], [133, 331], [332, 417], [...",[{'annotations': {'nda-11': {'choice': 'NotMen...,search-pdf,https://www.ungm.org/UNUser/Documents/Download...
1,86,CopAcc_NDA-and-ToP-Mentors_2.0_2017.pdf,NON-DISCLOSURE AGREEMENT AND TERMS OF PARTICIP...,"[[0, 51], [52, 68], [69, 222], [223, 358], [35...",[{'annotations': {'nda-11': {'choice': 'NotMen...,search-pdf,http://www.copernicus-masters.com/wp-content/u...
2,87,Cyber Mutual Assistance NDA.pdf,Mutual Non-Disclosure and Use of Information A...,"[[0, 54], [55, 99], [100, 400], [401, 423], [4...",[{'annotations': {'nda-11': {'choice': 'NotMen...,search-pdf,https://www.electricitysubsector.org/-/media/F...
3,88,DBT%20Mutual%20NDA.pdf,MUTUAL NON-DISCLOSURE AGREEMENT\nThis Non-Disc...,"[[0, 31], [32, 142], [142, 276], [277, 440], [...",[{'annotations': {'nda-11': {'choice': 'NotMen...,search-pdf,http://www.databoiler.com/index_htm_files/DBT%...
4,89,Data Use Agreement New York City.pdf,Data Use And Non-Disclosure Agreement\nBetween...,"[[0, 28], [28, 37], [38, 45], [46, 103], [104,...",[{'annotations': {'nda-11': {'choice': 'NotMen...,search-pdf,http://www.astho.org/Programs/Infectious-Disea...


### Sentence Tokenisation and Removing "\n"

In [ ]:
sentences = []
for i in range(len(df.text)):
    full_sent = nltk.sent_tokenize(df.text[i])
    for i in full_sent:
        for j in i:
            if(not j.isalpha()):
                i = i.replace(j, " ")
        if(len(i)<30 and len(i)>700):
            continue
        else:
            i = i.replace("\n", " ")
            sentences+=[i]



### Generating N-grams from sentences

In [ ]:
from nltk.util import ngrams
from nltk.tokenize import word_tokenize

n_gram_sentence = "The Parties have entered into this Agreement to facilitate the transfer of information between them and or one or more of their Affiliates in order for the Parties to evaluate whether or not to pursue a potential business opportunity or collaboration with respect to the development and commercialization of Prosensa s products including  but not limited to  Drisapersen  the  Purpose    and solely for that Purpose  the Parties have disclosed or may disclose to each other information that is proprietary and or confidential to the Disclosing Party which it desires be treated as confidential "
n_gram_sentence

tokenized_sentence = word_tokenize(n_gram_sentence)
output = list(ngrams(tokenized_sentence, 15))

print(output[0])
print(output[len(output) - 20])

('The', 'Parties', 'have', 'entered', 'into', 'this', 'Agreement', 'to', 'facilitate', 'the', 'transfer', 'of', 'information', 'between', 'them')
('Purpose', 'and', 'solely', 'for', 'that', 'Purpose', 'the', 'Parties', 'have', 'disclosed', 'or', 'may', 'disclose', 'to', 'each')


In [ ]:
sentences1 = sentences

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
en_stopwords = set(stopwords.words('english'))

for i in range(len(sentences)):
    tokenized_text = word_tokenize(sentences[i])
    filtered_text = [token for token in tokenized_text if token not in en_stopwords]
    sentences[i] = " ".join(filtered_text)
print(sentences[10])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rohit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


The Recipient acknowledges UNHCR hereto makes representation warranty express implied accuracy completeness Confidential Information


In [ ]:
try:
    while True:
        sentences.remove('')
except ValueError:
    pass

In [ ]:
ctr=0
for i in sentences:
    if(len(i)<30):
        sentences.remove(i)
        ctr+=1
        continue



In [ ]:
ctr

2474

In [ ]:
df1 = pd.DataFrame(data = sentences, columns = ["text"])

df1.to_excel("E:/Course work/NLP/contract-nli/contract-nli/a.xlsx")

In [ ]:
df1.head()

,text
0,NON DISCLOSURE AND CONFIDENTIALITY AGREEMENT T...
1,The Discloser Recipient also referred collecti...
2,RECITALS WHEREAS connection RFP Request Propos...
3,Confidential Information whenever used Agreeme...
4,The Confidential Information delivered otherwi...


In [ ]:
train_df = pd.read_excel('E:/Course work/NLP/labelled_dataset.xlsx')

NameError: ignored

In [ ]:
train_df.head()

,text,label
0,The Confidential Information delivered otherwi...,P
1,The Recipient may disclose Confidential Inform...,P
2,The Recipient shall precluded disclosing Confi...,O
3,The Recipient agrees indemnify UNHCR respect e...,O
4,Nothing Agreement shall construed obligating P...,O


In [ ]:
train_df = train_df.dropna()

In [ ]:
train_df["label"].value_counts()

P    286
O    245
F     88
Name: label, dtype: int64

In [ ]:
possible_labels = train_df.label.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'P': 0, 'O': 1, 'F': 2}

In [ ]:
train_df['label'] = train_df.label.replace(label_dict)

NameError: ignored